In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/demo-0/labeledTrainData.tsv
/kaggle/input/test-demo1/testData.tsv


In [2]:
import pandas as pd
train = pd.read_csv("/kaggle/input/demo-0/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [3]:
train

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone int..."
24996,"""5064_1""",0,"""I don't believe they made this film. Complete..."
24997,"""10905_3""",0,"""Guy is a loser. Can't get girls, needs to bui..."
24998,"""10194_3""",0,"""This 30 minute documentary Buñuel made in the..."


In [4]:
from bs4 import BeautifulSoup    
import re
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
import warnings
# 禁用特定警告
warnings.filterwarnings("ignore", category=UserWarning)
# 在执行可能触发警告的代码之前添加上述代码片段


In [5]:
def review_to_words(raw_review):
    # 函数将原始评论转换为单词字符串
    # 输入是一个字符串（原始电影评论）
    # 输出是一个字符串（预处理过的电影评论）
    # 1. 移除HTML
    review_text = BeautifulSoup(raw_review).get_text()
    # 2. 移除非字母字符
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    # 3. 转换为小写，分割成单个单词
    words = letters_only.lower().split()
    # 4. 在Python中，搜索集合比搜索列表要快得多，因此将停用词转换为集合
    stops = set(stopwords.words("english"))
    # 5. 移除停用词
    meaningful_words = [w for w in words if not w in stops]
    # 6. 将单词重新组合成一个由空格分隔的字符串，并返回结果
    return(" ".join(meaningful_words))
    #移除HTML：使用 BeautifulSoup 库从原始评论中移除所有的HTML标签，只留下纯文本。

#移除非字母字符：使用正则表达式通过 re.sub 函数替换掉所有非字母字符，只保留字母，并将它们转换为小写。

#转换为小写，分割成单个单词：将文本转换为小写，然后分割成单词列表。

#创建停用词集合：使用 nltk 库中的 stopwords 集合，创建一个停用词的集合。停用词是指在文本分析中通常会被忽略的词，如“the”、“is”、“and”等，因为它们对于理解文本的含义贡献不大。

#移除停用词：从单词列表中移除所有的停用词。

#将单词重新组合：将清洗后的单词列表重新组合成一个由空格分隔的单个字符串。

In [6]:
#所有
# 根据数据框列的大小获取评论数量
#num_reviews = train["review"].size

# 初始化一个空列表来保存清洗后的评论
clean_train_reviews = []
# 循环遍历每个评论；创建一个从0到电影评论列表长度的索引i
for i in range(0, len(train["review"])):
    # 为我们的函数调用每一个评论，并把结果添加到清洗后的评论列表中
    clean_train_reviews.append(review_to_words(train["review"][i]))

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
transfer=CountVectorizer(analyzer="word",  
                             tokenizer=None,  
                             preprocessor=None, 
                             stop_words=None,  
                             max_features=5000) 
#特征工程
train_data_features=transfer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

In [8]:
print(train_data_features.shape)

(25000, 5000)


In [9]:

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(train_data_features, train["sentiment"])

In [10]:
test = pd.read_csv("/kaggle/input/test-demo1/testData.tsv", header=0, delimiter="\t",  
                   quoting=3)


In [11]:
num_reviews = len(test["review"])
clean_test_reviews = []

In [12]:
for i in range(0, len(test["review"])):
    # 为我们的函数调用每一个评论，并把结果添加到清洗后的评论列表中
    clean_test_reviews.append(review_to_words(train["review"][i]))

In [13]:
test_data_features = transfer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [14]:
result = rf.predict(test_data_features)

In [15]:
output = pd.DataFrame(data={"id": test["id"], "sentiment": result})

In [16]:
output.to_csv("Bag_of_Words_model.csv", index=False, quoting=3)